In [93]:
!pip install torch torchvision

In [94]:
%matplotlib inline

Pytorch has two different data objects:




In [95]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html), all of which include datasets. For this tutorial, we will be using a TorchVision dataset. 



In [96]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
  root='data',
  train=True, #False for test dataset
  download=True,
  transform=ToTensor())

test_data = datasets.FashionMNIST(
  root='data',
  train=False, #False for test dataset
  download=True,
  transform=ToTensor())

We pass the Dataset as an argument to DataLoader to that wraps an iterable over the dataset, and supports automatic batching, sampling, shuffling, and multiprocess data loading.



In [97]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


--------------




To create a neural net in Pytorch you need to create a class that inherits the nn.Module object. You then define layers in the init function and create a function for the forward pass through the network. You can also put the model on GPU is you have one to use:



In [98]:
import torch
from torch import nn

class JacobsPytorchNN(nn.Module):
  def __init__(self):
    super(JacobsPytorchNN, self).__init__()
    self.flatten = nn.Flatten()
    self.network_architecture = nn.Sequential(
      nn.Linear(28*28,512),
      nn.ReLU(),
      nn.Linear(512,512),
      nn.ReLU(),
      nn.Linear(512,10),
      nn.ReLU()
    )
	
  def forward(self, x):
    x = self.flatten(x)
    logits = self.network_architecture(x)
    return logits

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
  
model = JacobsPytorchNN().to(device)

Using cpu device


--------------




To train we need a loss function and optimizer algo:



In [99]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

First we need to define a training loop & a evaluation function to get eval metric on test set:

In [100]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # Compute Loss
    pred = model(X)
    loss = loss_fn(pred, y)
    
    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
      
def evaluation(dataloader, model, loss_fn, optimizer):
  size = len(dataloader)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Next we write our training loop using the functions above:


In [101]:
num_epochs = 5
for i in range(num_epochs):
  print(f"Epoch {i+1}\n------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  evaluation(test_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
------------------------
loss: 2.305695  [    0/60000]
loss: 1.934930  [ 6400/60000]
loss: 1.557467  [12800/60000]
loss: 1.686610  [19200/60000]
loss: 1.184716  [25600/60000]
loss: 1.160299  [32000/60000]
loss: 1.193254  [38400/60000]
loss: 1.170361  [44800/60000]
loss: 1.193878  [51200/60000]
loss: 1.035588  [57600/60000]
Test Error: 
 Accuracy: 3514.0%, Avg loss: 1.170203 

Epoch 2
------------------------
loss: 0.995368  [    0/60000]
loss: 1.153615  [ 6400/60000]
loss: 1.084735  [12800/60000]
loss: 1.469074  [19200/60000]
loss: 1.037707  [25600/60000]
loss: 1.038368  [32000/60000]
loss: 1.074881  [38400/60000]
loss: 1.143590  [44800/60000]
loss: 1.129074  [51200/60000]
loss: 0.925802  [57600/60000]
Test Error: 
 Accuracy: 3628.7%, Avg loss: 1.105228 

Epoch 3
------------------------
loss: 0.892823  [    0/60000]
loss: 1.062170  [ 6400/60000]
loss: 1.023252  [12800/60000]
loss: 1.400670  [19200/60000]
loss: 0.987280  [25600/60000]
loss: 1.009968  [32000/60000]
loss: 1.03581

--------------




Loading & Saving Models
-------------


Loading models:

In [102]:
model = JacobsPytorchNN()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Saving models:

In [103]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


Inference
----------------------------



Making predictions on one new sample:


In [104]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
